We classify all census knots that share the same 0-surgery. (Here the 0-surgery is measured with respect to the Seifert framing.)

In [1]:
import snappy
import time
import csv
import itertools

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=index,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0

def subgroups_of_order_n(M,n):
    '''
    Returns the number of subgroups of order n of the fundamental group of M.
    Warning: Works only fast for small n.
    '''
    return len(snappy.Manifold(M).covers(n))

def subgroups_up_to_order_k(M,k):
    '''
    Returns the vector containing the number of subgroups of order n of the fundamental group of M for n=2,...,k.
    Warning: Works only fast for small k.
    '''
    vector=[]
    for n in range(2,k+1):
        vector.append(subgroups_of_order_n(M,n))
    return vector

def better_is_isometric_to(X,Y,return_isometries=False,index=100,try_hard=False):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y have different homologies.
     """ 
    if return_isometries==False:
        w='unclear'
        if X.homology()!=Y.homology():
            return False
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w==True:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=10,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=10,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,index=100,try_hard=False)
                    if w==True:
                        return w
        return 'unclear'
    if return_isometries==True:
        w=False
        if X.homology()!=Y.homology():
            return []
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y,return_isometries)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w!=False:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=10,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=10,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,return_isometries,index=100,try_hard=False)
                    if w!=False:
                        return w
        return []

First, we sort the knots into groups with the same Alexander polynomials.

In [2]:
start_time = time.time()
ALEX_GROUP=[]
alexander_polynomials=[]

for K in snappy.CensusKnots():
    f=K.alexander_polynomial()
    if f not in alexander_polynomials:
        alexander_polynomials.append(f)
        ALEX_GROUP.append([f,[K.identify()[0].name()]])
    else:
        for [pol,knots] in ALEX_GROUP:
            if pol==f:
                knots.append(K.identify()[0].name())

print('Number of different Alexander polynomials:',len(alexander_polynomials))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

Number of different Alexander polynomials: 1135
Time taken: 1.7811740279197692 minutes 


In [3]:
ALEX_GROUPS_WITH_MORE_THAN_ONE_KNOT=[x for x in ALEX_GROUP if len(x[1])>1]

In [4]:
len(ALEX_GROUPS_WITH_MORE_THAN_ONE_KNOT)

74

In [5]:
ALEX_GROUPS_WITH_MORE_THAN_ONE_KNOT

[[a^2 - 3*a + 1, ['m004', 't11898']],
 [2*a^2 - 3*a + 2, ['m015', 't10416']],
 [2*a^2 - 5*a + 2, ['m032', 'm372', 's879', 'v2861', 't08306', 'o9_22066']],
 [3*a^2 - 5*a + 3, ['m053', 'v3169']],
 [4*a^2 - 7*a + 4, ['m094', 's648', 'v2001', 'v2743']],
 [a^4 - 2*a^3 + a^2 - 2*a + 1, ['m199', 'o9_34801']],
 [a^4 - a^3 + a^2 - a + 1, ['m201', 'o9_34818']],
 [a^4 - 2*a^3 + 3*a^2 - 2*a + 1,
  ['m222', 's704', 'v2543', 't07599', 'o9_20894']],
 [a^4 - a^3 - a^2 - a + 1, ['m224', 'v3093', 't11814']],
 [4*a^2 - 9*a + 4, ['s016', 's726', 'o9_34700']],
 [5*a^2 - 9*a + 5, ['s023', 'o9_31990']],
 [a^6 - a^5 + a^3 - a + 1, ['s188', 'o9_22337', 'o9_41995']],
 [a^6 - a^5 - a^3 - a + 1, ['s194', 't10500']],
 [a^4 - a^2 + 1, ['s239', 't11441']],
 [a^6 - 2*a^5 + a^3 - 2*a + 1, ['s451', 'o9_30542', 'o9_40291']],
 [a^4 + a^3 - 3*a^2 + a + 1, ['s580', 't09927']],
 [a^4 - 3*a^3 + 5*a^2 - 3*a + 1, ['s912', 'o9_40568']],
 [5*a^2 - 11*a + 5, ['v0016', 'v2257', 'o9_37795']],
 [6*a^2 - 11*a + 6, ['v0025', 'v2284', 

For each of these 74 groups we check Dunfiled's list for exceptional surgeries and if it is hyperbolic we build the 0-surgery and compare their verified volumes.

In [6]:
exceptional_fillings=[]
with open('exceptional_fillings.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        exceptional_fillings.append(row)
len(exceptional_fillings)

205823

In [7]:
start_time = time.time()

ALEX_GROUPS_WITH_VOLUMENS=[]
ALEX_GROUPS_WITH_REGINA_NAMES=[]

for x in ALEX_GROUPS_WITH_MORE_THAN_ONE_KNOT:
    volumes=[]
    names=[]
    for name in x[1]:
        K=snappy.Manifold(name)
        zero_slope=K.homological_longitude()
        hyperbolic=True
        for x in exceptional_fillings:
            if x[-2]==name:
                if x[-1]==str(zero_slope):
                    names.append([name,x[2]])
                    hyperbolic=False
                if x[-1]==str((-zero_slope[0],-zero_slope[1])):
                    names.append([name,x[2]])
                    hyperbolic=False
        if hyperbolic:
            K.dehn_fill(zero_slope)
            vol=better_volume(K,try_hard=True)
            volumes.append([name,vol])
    if len(volumes)>1:
        volumes.sort(key=lambda x: x[1])
        ALEX_GROUPS_WITH_VOLUMENS.append(volumes)
    if len(names)>1:
        ALEX_GROUPS_WITH_REGINA_NAMES.append(names)

print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

Time taken: 1.0673562606175742 minutes 


To work with these lists it will be actually easier to have combinations with all possible pairs:

In [8]:
helplist=[]
for x in ALEX_GROUPS_WITH_REGINA_NAMES:
    for a,b in itertools.combinations(x, 2):
        helplist.append([a, b])
ALEX_GROUPS_WITH_REGINA_NAMES=helplist
ALEX_GROUPS_WITH_REGINA_NAMES

[[['m004', 'T x I / [ 2,1 | 1,1 ]'],
  ['t11898',
   "JSJ([('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('SFSpace', 'SFS [Or, g=0 + 3 punctures: (1,1)]')])"]],
 [['m015', 'SFS [A: (2,1)] / [ 0,1 | 1,-1 ]'],
  ['t10416', 'SFS [A: (2,1)] / [ 3,11 | 2,7 ]']],
 [['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
  ['m372', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]']],
 [['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
  ['s879', 'SFS [A: (2,1)] / [ 2,5 | 1,2 ]']],
 [['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
  ['v2861', 'SFS [A: (2,1)] / [ 2,7 | 1,3 ]']],
 [['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
  ['t08306', 'SFS [A: (2,1)] / [ 2,9 | 1,4 ]']],
 [['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
  ['o9_22066', 'SFS [A: (2,1)] / [ 2,11 | 1,5 ]']],
 [['m372', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]'],
  ['s879', 'SFS [A: (2,1)] / [ 2,5 | 1,2 ]']],
 [['m372', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]'],
  ['v2861', 'SFS [A: (2,1)] / [ 2,7 | 1,3 ]']],
 [['m372', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]'],
  ['t08306', 'SFS [A: (2,1)] / 

In [9]:
helplist=[]
for x in ALEX_GROUPS_WITH_VOLUMENS:
    for a,b in itertools.combinations(x, 2):
        helplist.append([a, b])
ALEX_GROUPS_WITH_VOLUMENS=helplist
ALEX_GROUPS_WITH_VOLUMENS

[[['m199', 3.177293278601?], ['o9_34801', 3.177293278600?]],
 [['m224', 2.78183391240?], ['v3093', 2.781833912396?]],
 [['m224', 2.78183391240?], ['t11814', 5.64120269706?]],
 [['v3093', 2.781833912396?], ['t11814', 5.64120269706?]],
 [['s194', 3.50909036378?], ['t10500', 6.577389902878?]],
 [['s451', 4.48795616567?], ['o9_30542', 6.33580908350?]],
 [['s451', 4.48795616567?], ['o9_40291', 7.57772895633?]],
 [['o9_30542', 6.33580908350?], ['o9_40291', 7.57772895633?]],
 [['s580', 4.408180344705?], ['t09927', 4.55591888988?]],
 [['s912', 4.059766425639?], ['o9_40568', 5.67188354357?]],
 [['v0329', 3.82422316958?], ['o9_23786', 4.724193070534?]],
 [['v0329', 3.82422316958?], ['o9_29669', 7.010981552339?]],
 [['o9_23786', 4.724193070534?], ['o9_29669', 7.010981552339?]],
 [['v0535', 3.56139987499?], ['v0960', 4.639202471219?]],
 [['v0600', 4.393635924020?], ['o9_40171', 7.85708783364?]],
 [['v1063', 4.613255212924?], ['o9_33263', 7.33009980619?]],
 [['v1935', 5.389888525936?], ['o9_38471',

We start with the exceptional 0-surgeries.

We distinguish them by using the number of subgroups in the fundamental group.

In [10]:
start_time = time.time()

might_be_equal=[]

for [[knot1,recog1],[knot2,recog2]] in ALEX_GROUPS_WITH_REGINA_NAMES:
    K1=snappy.Manifold(knot1)
    K1.dehn_fill(K1.homological_longitude())
    print(knot1,recog1)
    vec1=subgroups_up_to_order_k(K1,6)
    print(vec1)
    K2=snappy.Manifold(knot2)
    K2.dehn_fill(K2.homological_longitude())
    print(knot2,recog2)
    vec2=subgroups_up_to_order_k(K2,6)
    print(vec2)
    if vec1==vec2:
        might_be_equal.append([[knot1,recog1],[knot2,recog2]])
    print('---------------')

print('Number of possibel equal exceptional 0-surgeries:',len(might_be_equal))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

m004 T x I / [ 2,1 | 1,1 ]
[1, 1, 2, 2, 5]
t11898 JSJ([('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('SFSpace', 'SFS [Or, g=0 + 3 punctures: (1,1)]')])
[1, 1, 2, 11, 56]
---------------
m015 SFS [A: (2,1)] / [ 0,1 | 1,-1 ]
[1, 1, 1, 1, 3]
t10416 SFS [A: (2,1)] / [ 3,11 | 2,7 ]
[1, 1, 1, 1, 3]
---------------
m032 SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
[1, 2, 1, 3, 3]
m372 SFS [A: (2,1)] / [ -1,3 | 1,-2 ]
[1, 5, 4, 6, 24]
---------------
m032 SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
[1, 2, 1, 3, 3]
s879 SFS [A: (2,1)] / [ 2,5 | 1,2 ]
[1, 2, 1, 8, 18]
---------------
m032 SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
[1, 2, 1, 3, 3]
v2861 SFS [A: (2,1)] / [ 2,7 | 1,3 ]
[1, 2, 1, 3, 3]
---------------
m032 SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
[1, 2, 1, 3, 3]
t08306 SFS [A: (2,1)] / [ 2,9 | 1,4 ]
[1, 5, 4, 6, 24]
---------------
m032 SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
[1, 2, 1, 3, 3]
o9_22066 SFS [A: (2,1)] / [ 2,11 | 1,5 ]
[1, 2, 1, 3, 3]
---------------
m372 SFS [A: (2,1)] / [ -1,3 | 1,-2 ]
[1, 5, 4, 6, 24]
s879 SFS [A: (2,1)] / [ 

t11418 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])
[1, 1, 1, 6, 11]
---------------
v1971 SFS [D: (2,1) (2,1)] U/m SFS [D: (5,2) (5,3)], m = [ 0,1 | 1,0 ]
[1, 1, 1, 8, 25]
o9_35240 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])
[1, 1, 1, 6, 11]
---------------
v1971 SFS [D: (2,1) (2,1)] U/m SFS [D: (5,2) (5,3)], m = [ 0,1 | 1,0 ]
[1, 1, 1, 8, 25]
o9_36459 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])
[1, 1, 1, 6, 11]
---------------
v1971 SFS [D: (2,1) (2,1)] U/m SFS [D: (5,2) (5,3)], m = [ 0,1 | 1,0 ]
[1, 1, 1, 8, 25]
o9_37618 SFS [D: (2,1) (2,1)] U/m SFS [D: (5,2) (5,3)], m = [ -2,3 | -1,2 ]
[1, 1, 1, 20, 85]
---------------
t10887 SFS [D: (2,1) (2,1)] U/m SFS [D: (5,2) (5,3)], m = [ -1,2 | 0,1 ]
[1, 1, 1, 12, 53]
t11418 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])
[1, 1, 1, 6, 11]
---------------
t10887 SFS [D: (2,1) (2,1)] U/m SFS [D: (5,2) (5,3)], m = [ -1,2 | 0,1 ]
[1, 1, 1, 12, 53]
o9_35240 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperboli

t07734 JSJ([('SFSpace', 'SFS [A: (3,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])
[1, 1, 2, 1, 3]
o9_36955 JSJ([('SFSpace', 'SFS [A: (3,4)]'), ('SFSpace', 'SFS [A: (3,4)]')])
[1, 1, 2, 1, 3]
---------------
o9_00017 SFS [A: (7,1)] / [ 0,1 | 1,-2 ]
[1, 1, 2, 3, 5]
o9_24293 SFS [A: (7,3)] / [ 2,5 | 1,2 ]
[1, 1, 2, 3, 5]
---------------
o9_10654 SFS [A: (10,3)] / [ -1,3 | 1,-2 ]
[1, 2, 4, 10, 48]
o9_13141 JSJ([('SFSpace', 'SFS [A: (2,5)]'), ('SFSpace', 'SFS [A: (5,6)]')])
[1, 2, 1, 1, 3]
---------------
o9_10654 SFS [A: (10,3)] / [ -1,3 | 1,-2 ]
[1, 2, 4, 10, 48]
o9_19831 JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (5,7)]')])
[1, 2, 1, 7, 33]
---------------
o9_13141 JSJ([('SFSpace', 'SFS [A: (2,5)]'), ('SFSpace', 'SFS [A: (5,6)]')])
[1, 2, 1, 1, 3]
o9_19831 JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (5,7)]')])
[1, 2, 1, 7, 33]
---------------
o9_27542 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])
[1, 1, 1, 1, 1]
o9_33568 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hy

In [11]:
start_time = time.time()

probably_equal=[]

for [[knot1,recog1],[knot2,recog2]] in might_be_equal:
    K1=snappy.Manifold(knot1)
    K1.dehn_fill(K1.homological_longitude())
    print(knot1,recog1)
    vec1=subgroups_up_to_order_k(K1,7)
    print(vec1)
    K2=snappy.Manifold(knot2)
    K2.dehn_fill(K2.homological_longitude())
    print(knot2,recog2)
    vec2=subgroups_up_to_order_k(K2,7)
    print(vec2)
    if vec1==vec2:
        probably_equal.append([[knot1,recog1],[knot2,recog2]])
    print('---------------')

print('Number of probably equal exceptional 0-surgeries:',len(probably_equal))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

m015 SFS [A: (2,1)] / [ 0,1 | 1,-1 ]
[1, 1, 1, 1, 3, 5]
t10416 SFS [A: (2,1)] / [ 3,11 | 2,7 ]
[1, 1, 1, 1, 3, 5]
---------------
m032 SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
[1, 2, 1, 3, 3, 3]
v2861 SFS [A: (2,1)] / [ 2,7 | 1,3 ]
[1, 2, 1, 3, 3, 38]
---------------
m032 SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
[1, 2, 1, 3, 3, 3]
o9_22066 SFS [A: (2,1)] / [ 2,11 | 1,5 ]
[1, 2, 1, 3, 3, 3]
---------------
m372 SFS [A: (2,1)] / [ -1,3 | 1,-2 ]
[1, 5, 4, 6, 24, 9]
t08306 SFS [A: (2,1)] / [ 2,9 | 1,4 ]
[1, 5, 4, 6, 24, 9]
---------------
v2861 SFS [A: (2,1)] / [ 2,7 | 1,3 ]
[1, 2, 1, 3, 3, 38]
o9_22066 SFS [A: (2,1)] / [ 2,11 | 1,5 ]
[1, 2, 1, 3, 3, 3]
---------------
m201 SFS [S2: (2,1) (5,1) (10,-7)]
[1, 1, 1, 3, 4, 2]
o9_34818 SFS [S2: (2,1) (5,1) (10,-7)]
[1, 1, 1, 3, 4, 2]
---------------
s726 JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])
[1, 1, 1, 1, 11, 23]
o9_34700 JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,3)]')])
[1, 1, 1, 1, 11, 56]
---------------
s023 SFS

In [12]:
probably_equal

[[['m015', 'SFS [A: (2,1)] / [ 0,1 | 1,-1 ]'],
  ['t10416', 'SFS [A: (2,1)] / [ 3,11 | 2,7 ]']],
 [['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
  ['o9_22066', 'SFS [A: (2,1)] / [ 2,11 | 1,5 ]']],
 [['m372', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]'],
  ['t08306', 'SFS [A: (2,1)] / [ 2,9 | 1,4 ]']],
 [['m201', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
  ['o9_34818', 'SFS [S2: (2,1) (5,1) (10,-7)]']],
 [['s239', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]'],
  ['t11441',
   'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]']],
 [['v0025', 'SFS [A: (6,5)] / [ 0,1 | 1,-1 ]'],
  ['v2284',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"]],
 [['o9_33067',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"],
  ['o9_34323',
   "JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (3,5)]')])"]],
 [['v0595', 'SFS [S2: (2,1) (7,2) (14,-11)]'],
  ['t12120', 'SFS [S2: (2,1) (7,2) (14,-11)]']],
 [['v2362',
   "JSJ([('SFSpace', 

Some of those we see directly that they are equal. For the others we try a bit harder to distinguish them.

In [13]:
equal_exceptional_surgeries=[]

equal_exceptional_surgeries.append([['m201', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
  ['o9_34818', 'SFS [S2: (2,1) (5,1) (10,-7)]']])
probably_equal.remove([['m201', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
  ['o9_34818', 'SFS [S2: (2,1) (5,1) (10,-7)]']])

equal_exceptional_surgeries.append([['s239', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]'],
  ['t11441',
   'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]']])
probably_equal.remove([['s239', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]'],
  ['t11441',
   'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]']])

equal_exceptional_surgeries.append([['v0595', 'SFS [S2: (2,1) (7,2) (14,-11)]'],
  ['t12120', 'SFS [S2: (2,1) (7,2) (14,-11)]']])
probably_equal.remove([['v0595', 'SFS [S2: (2,1) (7,2) (14,-11)]'],
  ['t12120', 'SFS [S2: (2,1) (7,2) (14,-11)]']])

len(probably_equal)

9

In [14]:
start_time = time.time()

probably_equal2=[]

for [[knot1,recog1],[knot2,recog2]] in probably_equal:
    K1=snappy.Manifold(knot1)
    K1.dehn_fill(K1.homological_longitude())
    print(knot1,recog1)
    vec1=subgroups_of_order_n(K1,8)
    print(vec1)
    K2=snappy.Manifold(knot2)
    K2.dehn_fill(K2.homological_longitude())
    print(knot2,recog2)
    vec2=subgroups_of_order_n(K2,8)
    print(vec2)
    if vec1==vec2:
        probably_equal2.append([[knot1,recog1],[knot2,recog2]])
    print('---------------')

print('Number of probably equal exceptional 0-surgeries:',len(probably_equal2))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

m015 SFS [A: (2,1)] / [ 0,1 | 1,-1 ]
8
t10416 SFS [A: (2,1)] / [ 3,11 | 2,7 ]
8
---------------
m032 SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
5
o9_22066 SFS [A: (2,1)] / [ 2,11 | 1,5 ]
5
---------------
m372 SFS [A: (2,1)] / [ -1,3 | 1,-2 ]
41
t08306 SFS [A: (2,1)] / [ 2,9 | 1,4 ]
41
---------------
v0025 SFS [A: (6,5)] / [ 0,1 | 1,-1 ]
5
v2284 JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])
5
---------------
o9_33067 JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])
453
o9_34323 JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (3,5)]')])
453
---------------
v2362 JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])
5
t00017 SFS [A: (6,1)] / [ 0,1 | 1,-2 ]
5
---------------
t05252 SFS [A: (8,3)] / [ -1,3 | 1,-2 ]
193
o9_19326 JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (4,5)]')])
193
---------------
o9_27542 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])
1
o9_33568 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 

However, if we look more carefully at the regina names, we see that several of those are not diffeomorphic. We can distinguish two Seifert fibered spaces over the annulus by the traces of their gluing maps. Moreover, if we have one manifold a Seifert fibered space over an annulus with a single exceptional fiber and another space a JSJ manifold built by gluing together two Seifert fibered spaces over trhe annulus each with a single exceptional fiber, then the cannot be diffeomorphic. We remove these pairs from the list. And work a bit harder with the remaining examples.

In [14]:
probably_equal.remove([['m015', 'SFS [A: (2,1)] / [ 0,1 | 1,-1 ]'],
  ['t10416', 'SFS [A: (2,1)] / [ 3,11 | 2,7 ]']])
probably_equal.remove([['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
  ['o9_22066', 'SFS [A: (2,1)] / [ 2,11 | 1,5 ]']])
probably_equal.remove([['m372', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]'],
  ['t08306', 'SFS [A: (2,1)] / [ 2,9 | 1,4 ]']])
probably_equal.remove([['v0025', 'SFS [A: (6,5)] / [ 0,1 | 1,-1 ]'],
  ['v2284',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"]])
probably_equal.remove([['v2362',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])"],
  ['t00017', 'SFS [A: (6,1)] / [ 0,1 | 1,-2 ]']])
probably_equal.remove([['t05252', 'SFS [A: (8,3)] / [ -1,3 | 1,-2 ]'],
  ['o9_19326',
   "JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (4,5)]')])"]])

In [15]:
probably_equal

[[['o9_33067',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"],
  ['o9_34323',
   "JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (3,5)]')])"]],
 [['o9_27542', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])"],
  ['o9_33568', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])"]],
 [['o9_31828', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])"],
  ['o9_37768', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])"]]]

In [16]:
start_time = time.time()

probably_equal2=[]

for [[knot1,recog1],[knot2,recog2]] in probably_equal[1:]:
    K1=snappy.Manifold(knot1)
    K1.dehn_fill(K1.homological_longitude())
    print(knot1,recog1)
    vec1=subgroups_of_order_n(K1,9)
    print(vec1)
    K2=snappy.Manifold(knot2)
    K2.dehn_fill(K2.homological_longitude())
    print(knot2,recog2)
    vec2=subgroups_of_order_n(K2,9)
    print(vec2)
    if vec1==vec2:
        probably_equal2.append([[knot1,recog1],[knot2,recog2]])
    print('---------------')

print('Number of probably equal exceptional 0-surgeries:',len(probably_equal2))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

o9_27542 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])
15
o9_33568 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])
15
---------------
o9_31828 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])
21
o9_37768 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])
21
---------------
Number of probably equal exceptional 0-surgeries: 2
Time taken: 0.05368261734644572 minutes 


TO DO: Use regina to compare?

In [17]:
start_time = time.time()

probably_equal2=[]

for [[knot1,recog1],[knot2,recog2]] in probably_equal[1:]:
    K1=snappy.Manifold(knot1)
    K1.dehn_fill(K1.homological_longitude())
    print(knot1,recog1)
    vec1=subgroups_of_order_n(K1,10)
    print(vec1)
    K2=snappy.Manifold(knot2)
    K2.dehn_fill(K2.homological_longitude())
    print(knot2,recog2)
    vec2=subgroups_of_order_n(K2,10)
    print(vec2)
    if vec1==vec2:
        probably_equal2.append([[knot1,recog1],[knot2,recog2]])
    print('---------------')

print('Number of probably equal exceptional 0-surgeries:',len(probably_equal2))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

o9_27542 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])
17
o9_33568 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])
17
---------------
o9_31828 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])
56
o9_37768 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])
56
---------------
Number of probably equal exceptional 0-surgeries: 2
Time taken: 0.829452900091807 minutes 


In [18]:
start_time = time.time()

probably_equal2=[]

for [[knot1,recog1],[knot2,recog2]] in probably_equal[1:]:
    K1=snappy.Manifold(knot1)
    K1.dehn_fill(K1.homological_longitude())
    print(knot1,recog1)
    vec1=subgroups_of_order_n(K1,11)
    print(vec1)
    K2=snappy.Manifold(knot2)
    K2.dehn_fill(K2.homological_longitude())
    print(knot2,recog2)
    vec2=subgroups_of_order_n(K2,11)
    print(vec2)
    if vec1==vec2:
        probably_equal2.append([[knot1,recog1],[knot2,recog2]])
    print('---------------')

print('Number of probably equal exceptional 0-surgeries:',len(probably_equal2))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

o9_27542 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])
19
o9_33568 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])
19
---------------
o9_31828 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])
52
o9_37768 JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])
52
---------------
Number of probably equal exceptional 0-surgeries: 2
Time taken: 10.703543718655904 minutes 


In summary we have found 3 pairs of equal exceptional 0-surgeries and 3 pairs that are probably equal. Here we need to work a bit harder. Maybe we can see the equality by doing explicit Kirby moves. All these knots have low unknotting number (maybe 1) so Kirby calculus might help here. We will do that later.

In [17]:
equal_exceptional_surgeries

[[['m201', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
  ['o9_34818', 'SFS [S2: (2,1) (5,1) (10,-7)]']],
 [['s239', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]'],
  ['t11441',
   'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]']],
 [['v0595', 'SFS [S2: (2,1) (7,2) (14,-11)]'],
  ['t12120', 'SFS [S2: (2,1) (7,2) (14,-11)]']]]

In [16]:
probably_equal

[[['o9_33067',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"],
  ['o9_34323',
   "JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (3,5)]')])"]],
 [['o9_27542', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])"],
  ['o9_33568', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')])"]],
 [['o9_31828', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])"],
  ['o9_37768', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm032')])"]]]

We continue to work with the hyperbolic surgeries. First we remove the pairs with different volumes.

In [18]:
pairs_with_same_volume=[]

for [[knot1,vol1],[knot2,vol2]] in ALEX_GROUPS_WITH_VOLUMENS:
    if vol1.overlaps(vol2):
        print([knot1,vol1])
        print([knot2,vol2])
        pairs_with_same_volume.append([[knot1,vol1],[knot2,vol2]])
        print('-----------------')
        
pairs_with_same_volume                    

['m199', 3.177293278601?]
['o9_34801', 3.177293278600?]
-----------------
['m224', 2.78183391240?]
['v3093', 2.781833912396?]
-----------------
['v3423', 4.415332477454?]
['v3536', 4.41533247746?]
-----------------
['v3423', 4.415332477454?]
['o9_42735', 4.41533247746?]
-----------------
['v3536', 4.41533247746?]
['o9_42735', 4.41533247746?]
-----------------
['t12607', 5.28793627053?]
['t12748', 5.28793627053?]
-----------------
['o9_41909', 5.83941124979?]
['o9_42515', 5.83941124979?]
-----------------
['v2508', 3.702897321857?]
['v3195', 3.702897321857?]
-----------------
['t11532', 4.90949251141?]
['t12200', 4.90949251141?]
-----------------
['t11532', 4.90949251141?]
['o9_43446', 4.909492511407?]
-----------------
['t12200', 4.90949251141?]
['o9_43446', 4.909492511407?]
-----------------
['o9_39433', 5.59095176819?]
['o9_40519', 5.59095176819?]
-----------------
['v2869', 4.97179004792?]
['t12388', 4.971790047912?]
-----------------
['t07281', 4.162020103638?]
['o9_34949', 4.16202

[[['m199', 3.177293278601?], ['o9_34801', 3.177293278600?]],
 [['m224', 2.78183391240?], ['v3093', 2.781833912396?]],
 [['v3423', 4.415332477454?], ['v3536', 4.41533247746?]],
 [['v3423', 4.415332477454?], ['o9_42735', 4.41533247746?]],
 [['v3536', 4.41533247746?], ['o9_42735', 4.41533247746?]],
 [['t12607', 5.28793627053?], ['t12748', 5.28793627053?]],
 [['o9_41909', 5.83941124979?], ['o9_42515', 5.83941124979?]],
 [['v2508', 3.702897321857?], ['v3195', 3.702897321857?]],
 [['t11532', 4.90949251141?], ['t12200', 4.90949251141?]],
 [['t11532', 4.90949251141?], ['o9_43446', 4.909492511407?]],
 [['t12200', 4.90949251141?], ['o9_43446', 4.909492511407?]],
 [['o9_39433', 5.59095176819?], ['o9_40519', 5.59095176819?]],
 [['v2869', 4.97179004792?], ['t12388', 4.971790047912?]],
 [['t07281', 4.162020103638?], ['o9_34949', 4.16202010364?]],
 [['t07281', 4.162020103638?], ['o9_39806', 4.162020103638?]],
 [['o9_34949', 4.16202010364?], ['o9_39806', 4.162020103638?]],
 [['t10974', 5.298314671809?

In [21]:
start_time = time.time()

probably_equal=[]

for [[knot1,recog1],[knot2,recog2]] in pairs_with_same_volume:
    K1=snappy.Manifold(knot1)
    K1.dehn_fill(K1.homological_longitude())
    print(knot1,vol1)
    vec1=subgroups_up_to_order_k(K1,6)
    print(vec1)
    K2=snappy.Manifold(knot2)
    K2.dehn_fill(K2.homological_longitude())
    print(knot2,vol2)
    vec2=subgroups_up_to_order_k(K2,6)
    print(vec2)
    if vec1==vec2:
        probably_equal.append([knot1,knot2])
    print('---------------')

print('Number of probably equal hyperbolic 0-surgeries:',len(probably_equal))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

m199 7.97594386118?
[1, 1, 2, 1, 3]
o9_34801 8.17492752434?
[1, 1, 2, 1, 3]
---------------
m224 7.97594386118?
[1, 2, 4, 1, 11]
v3093 8.17492752434?
[1, 2, 4, 1, 11]
---------------
v3423 7.97594386118?
[1, 1, 6, 4, 12]
v3536 8.17492752434?
[1, 1, 6, 4, 12]
---------------
v3423 7.97594386118?
[1, 1, 6, 4, 12]
o9_42735 8.17492752434?
[1, 1, 6, 4, 12]
---------------
v3536 7.97594386118?
[1, 1, 6, 4, 12]
o9_42735 8.17492752434?
[1, 1, 6, 4, 12]
---------------
t12607 7.97594386118?
[1, 1, 2, 3, 3]
t12748 8.17492752434?
[1, 1, 2, 3, 3]
---------------
o9_41909 7.97594386118?
[1, 1, 6, 4, 14]
o9_42515 8.17492752434?
[1, 1, 6, 4, 14]
---------------
v2508 7.97594386118?
[1, 1, 1, 3, 1]
v3195 8.17492752434?
[1, 1, 1, 3, 1]
---------------
t11532 7.97594386118?
[1, 1, 1, 5, 5]
t12200 8.17492752434?
[1, 1, 1, 5, 5]
---------------
t11532 7.97594386118?
[1, 1, 1, 5, 5]
o9_43446 8.17492752434?
[1, 1, 1, 5, 5]
---------------
t12200 7.97594386118?
[1, 1, 1, 5, 5]
o9_43446 8.17492752434?
[1, 1, 

So they all seem to be equal. We search for an actual isometry.

In [23]:
for y in probably_equal:
    print(y)
    K1=snappy.Manifold(y[0])
    K1.dehn_fill(K1.homological_longitude())
    K2=snappy.Manifold(y[1])
    K2.dehn_fill(K2.homological_longitude())
    print(K1.is_isometric_to(K2))
    print('------') 

['m199', 'o9_34801']
True
------
['m224', 'v3093']
True
------
['v3423', 'v3536']
True
------
['v3423', 'o9_42735']
True
------
['v3536', 'o9_42735']
True
------
['t12607', 't12748']
True
------
['o9_41909', 'o9_42515']
True
------
['v2508', 'v3195']
True
------
['t11532', 't12200']
True
------
['t11532', 'o9_43446']
True
------
['t12200', 'o9_43446']
True
------
['o9_39433', 'o9_40519']
True
------
['v2869', 't12388']
True
------
['t07281', 'o9_34949']
True
------
['t07281', 'o9_39806']
True
------
['o9_34949', 'o9_39806']
True
------
['t10974', 'o9_39967']
True
------
['t09735', 't11748']
True
------
['t09900', 'o9_34908']
True
------
['t09900', 'o9_43876']
True
------
['o9_34908', 'o9_43876']
True
------
['t11900', 'o9_40803']
True
------
['o9_33833', 'o9_37547']
True
------


In [24]:
equal_hyperbolic_surgeries=probably_equal
equal_hyperbolic_surgeries

[['m199', 'o9_34801'],
 ['m224', 'v3093'],
 ['v3423', 'v3536'],
 ['v3423', 'o9_42735'],
 ['v3536', 'o9_42735'],
 ['t12607', 't12748'],
 ['o9_41909', 'o9_42515'],
 ['v2508', 'v3195'],
 ['t11532', 't12200'],
 ['t11532', 'o9_43446'],
 ['t12200', 'o9_43446'],
 ['o9_39433', 'o9_40519'],
 ['v2869', 't12388'],
 ['t07281', 'o9_34949'],
 ['t07281', 'o9_39806'],
 ['o9_34949', 'o9_39806'],
 ['t10974', 'o9_39967'],
 ['t09735', 't11748'],
 ['t09900', 'o9_34908'],
 ['t09900', 'o9_43876'],
 ['o9_34908', 'o9_43876'],
 ['t11900', 'o9_40803'],
 ['o9_33833', 'o9_37547']]

We check the 4-genus and the s-invariants of these knots.

In [25]:
slice_genus=[]
with open('slice_top_bounds_23_08.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        slice_genus.append([row[0],row[1]])
len(slice_genus)

1267

In [30]:
s_invariants=[]
with open('sinvariants.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        s_invariants.append([row[0],row[1],row[2]])
len(s_invariants)

1332

In [31]:
for x in equal_hyperbolic_surgeries:
    print(x)
    for y in x:
        for z in slice_genus:
            if z[0]==y:
                break
        print('4-genus:',z)
        for w in s_invariants:
            if w[1]==y:
                print('s-invariant:',w[2])
                break
        
    print('-----------')

['m199', 'o9_34801']
4-genus: ['m199', '1']
s-invariant: 0
4-genus: ['o9_34801', '(1, 2)']
s-invariant: -2
-----------
['m224', 'v3093']
4-genus: ['m224', '1']
s-invariant: 0
4-genus: ['v3093', '(1, 2)']
s-invariant: 0
-----------
['v3423', 'v3536']
4-genus: ['v3423', '(0, 2)']
s-invariant: 0
4-genus: ['v3536', '0']
s-invariant: 0
-----------
['v3423', 'o9_42735']
4-genus: ['v3423', '(0, 2)']
s-invariant: 0
4-genus: ['o9_42735', '(0, 2)']
-----------
['v3536', 'o9_42735']
4-genus: ['v3536', '0']
s-invariant: 0
4-genus: ['o9_42735', '(0, 2)']
-----------
['t12607', 't12748']
4-genus: ['t12607', '(0, 2)']
s-invariant: 0
4-genus: ['t12748', '0']
s-invariant: 0
-----------
['o9_41909', 'o9_42515']
4-genus: ['o9_41909', '(0, 2)']
s-invariant: 0
4-genus: ['o9_42515', '(0, 2)']
s-invariant: 0
-----------
['v2508', 'v3195']
4-genus: ['v2508', '(0, 1)']
s-invariant: 0
4-genus: ['v3195', '0']
s-invariant: 0
-----------
['t11532', 't12200']
4-genus: ['t11532', '(0, 2)']
s-invariant: 0
4-genus: ['